In [1]:
P = 3221225473
F = GF(P)
F_gen = F.gen()
w = F.multiplicative_generator()
assert F_gen == 1

g = 1734477367 * F_gen
g_order = 8192

RS_factor = 8

gamma = g**RS_factor
gamma_order = sage.rings.integer.Integer(g_order / RS_factor)
trace_size = gamma_order

PR = F['x']

fib_1 = F_gen
last_fib_index = 1000
fib_last = F_gen * 2997353887
fib_constraint_max_deg = 1023 - 999
boundary_constraint_max_deg = 1023 - 1

N_FRI_QUERIES = 2  # change this to two and find bug in mask paths

In [2]:
import hashlib

def enc_and_hash(data):
    return hashlib.sha256(str(data).encode("utf-8")).digest()

class FiatShamir():
    def __init__(self, seed=None):
        if seed is None:
            self.last_hash = b""
        else:
            self.lash_hash = seed

        self.nonce = 0
    
    def get_random_int(self, data):
        self.last_hash = hashlib.sha256(self.last_hash + enc_and_hash(data) + enc_and_hash(self.nonce)).digest()
        self.nonce += 1
        return int(self.last_hash.hex(), 16)

In [4]:
proof_example = ['eval_merkle:570761f588817960dc6d717177881d9be50ae3b58d8bf7d3b3d2a0be8571fbce', 'cp_eval_merkle:04176464b55c32d8c16d5aba993d664aec8a3f336e534cfd2214b81f721730a5', 'cp_z:482443502', 'mask_z1:609728494', 'mask_z2:652591348', 'mask_z3:2801782619', 'oods_eval_merkle:dc6eddbd17ed77e0b6b6d848f2a90aa9ebf9a24d36c81ba1ba9295feb4589cf6', 'layer_root:958b0be2cc530d78b831f074c38ba49e2771f149d57f79ceaf6d36cf64d83184', 'layer_root:845728a598d02facc10a017b74d248f41e14948cee29f363b373a11f24ba94db', 'layer_root:d5e9e4c241a73e5004b64300b7d2f559a51e0ca443c040787fe8c39b74cf652f', 'layer_root:9cc197531e18bd6ac7a7b008f5011245ae13a16ef559b38507f31cb2b6e6f2b6', 'layer_root:2cd69bbe4bc79d42602f439f65ba758f1919520f0e466eb0aa31c66ef8798664', 'layer_root:3628fb42d66976512a98221509536385eab42df76e11971f5643d5c45665ded5', 'layer_root:cefe761f32adcd55dd57ccd9d38ca6e326ea501cfefac4f32ec2644a4b6ac4f3', 'layer_root:2e1c18785f302c5c2d6a2674d0349e4d183be4522dddea16bea1d937f5c985c0', 'layer_root:97db617edf09fa55845850ceb5672326ff221063f674f9521b3da4823494c7f5', 'last_layer_const:1747311642', 'trace_eval:1743475752', "path:['03959da8a7ca8c83308542539f578bd81e55d0a6b0bec0f35479b96d2a497910', '726c53f0c87f68ae8977b7d186626f13dd3e78f69bbe34e6386287a278f96017', '6bed7b7c3ef0a01c37356e58ccead5801122309be375d0415e8ff0369f04c378', '987b03006744eb1f0b7234e66ee44e3f483d2357c5eb4a6190737fcbcf367dad', 'c52e739c6e34c53de26136534d23ff6e4aef3fdcfbf7dc52cee0dca932375b0e', 'ea4f7cce668193e116f31c1eea9a7058ddb7eddb6ad67126e16dca653b13d1ee', '1bb98e595181a3d66c873829b67e173ead69c7abff49a842fea96304b4e7535c', '653ec024805b0f4cbb6a8547de772702ac58990801c0278ae41d86bbb1141ebf', '68196a35f42e9c18b6f0256dfee4228ba678b04812c601255ab01891c6bcacd0', '09bcb32f8f320b0104b0e76180a54a04b77feb815acc0ac0f46e5eeeca4011ce', 'c17eea82780d4f96c285c4d0a07358bc8bbbb4c521a32cb665401f54fe65e5c4', 'ed47ff0c8ce61c50430b42e6f5b2028becc1dca21c9c3f14ea92f860cae6a39b', '0bd2e2cbd66617f60fdf312d7db7e01d107f6856b02b7f900a1e484625057e47']", 'cp_eval:652424948', "path:['0ea7dd497602c3d3be835a3a4ca7fda8590af1465a3d76ac7085d5cea707624d', '2f0b448dcc34318b77a88d3a1807ef049b6bc26d42282cb31f046d0a51ce1963', 'ee5c8571ea0273f9eb8568589863085507877922ec2723ddf0b955f8ea4bb349', '1b38b1681e4eb6862e08c57023e619cb20a8f5bde815429453000eed22321e45', '1d0648a6aebc72077881bc43a04ce273b56e3cbbd5acf2a7b7b1d24f0a444594', 'b46a9b76e54e4cab34b07b07a3e5d5343743876b6715c9361ff60c8f58bdc46a', '9a723bab39be04a164d947b57b0b82180a873d650cf584484b9d40a3a140c780', '8a58809abd872d6666e6f782032bc26b81267a7b6eb1edcd9b4fbcc74eb84241', '3f8f09749ba409e1aa046b06fe82a63b07a5656673793448675f5ca4303597fe', '2064c3e282721c5aa0c3d1c711dd9d9c6128da36c301e5621e9d806e63702cba', '939b8cddc83ec8232592829973f871333a1510c0b338648e3b0661ecd77afeb7', 'ea41cbb921bb44951efcddd9103c57a752b19a0759ff3a4911b8a05af72697d6', '2ed24b8fbf362309123590d626e8068c76238aa795798bdcf1aa905b27bf5e2e']", 'layer_first_eval:3066045976', "path:['e6e76f52b03e5afbb5df89ab5c58fac775643627c78752177d2de970280f5a3e', '216413cfabf3a848f14cad0f88dc83518b62ac8b04c6b75d1bdf2bf4659d3665', '1970b460c853e0b1d0963e77eb313de7bbafd076a5408e95d8ac6d4910ed3848', '5ea06b52acbcdae4a596560f7fbafad9c7494c4d3f80728c10a8ddb2c36d1a63', 'f8f525eafe8cff1557624879e539b70a3725bc69e946708f938effa373fcfffc', 'be2b4c9928c69087a0aa49236813f3b2326721b29eea4bb8935ee9ea1e584d18', 'c595567cffba9024ef0c983d6bdf2a1a88f576b26565df09db0d7e0aa6665b3c', 'cab3f3c1a9d1f4a4b3f277f29a61f9ce258711fc24003121660bf85cf5d351d0', '3c434c3d50750bd8fd2a67d9aa1ab6ad62018163da48ab618334670c7d6bb6d9', 'bd6240db4318522eeffd43e62fdde6390ebbc2d396a22e0582cb574e0897f4c1', '1d088c57d29dec3233e587efe12d4004238fbf188d38527db3c382c87478568c', 'e0cfb338df902f5f65866b986caf30709e5bf78a1e6bfdc51acb5bda540ff960', 'a07ea554d0cdf7c9725322a6f73183bfc8b4fead29197cf6ebb1453ee54f9e72']", 'layer_second_eval:2183456114', "path:['539da894638921ce8ad18df77012b2d159db41e0dbd896a711bbb620a64a7ab8', '0cc818487548b876f25596218fe2b49e6bfc0e56aa6ecb776f80bc80c133d27a', '6ff55a51754f8230c79bdb147ead238a6c96fe2ad2758bfff5bae5a0267e44a2', '5e10dd9d57b72b8641d02bb9fff21f738990121f17454f6ce2233077fd90bd25', '04fc80869d2fc7429e4dc5fde8aa50d7f45448d3a865cc69c4b56b7276ac300f', 'dce483079f5e38a0606ea04111fd2b15722216452f438f403a0589b954da74ed', 'fbb02abdd601bd4913be754dd2c0b5d5058aa433574b9e471e1c83a11dbb3ad7', '79dc3d79817e65f1e24f2a2a8990c288e8da1f02ab0852c9d894f762e853d341', '3d83ba361b4de94915e700439b107bf1a86c1f4fe7a570a0ea19ad8ecb3af0e9', '52a4883032a588b07fe4d7dfd68dd57ff66ce53fe336c7eef07eb3afa5460bd1', '726332a175fcc8bdb44e5a07570542a1459544df17d8cc5e915d17353f44b70f', '430726819ab696bf936e97339871ca985081a9d71c9090c8a2cf52684c061d39', 'f91020be7cb8282ff9226196ef53e97d8fe4ba07e71e79c47d2380e607932e35']", 'layer_first_eval:938691637', "path:['dad6e9636b6e17e2ca1a0fdacb19861d43c48611d450f288ba99d59590e39c30', '95220b8660843fb25175e7a2097ee8b10f6fd3d84f26f9964a46dcf863334895', '5c6ec30706b2efb4ab7ebe8ad69dbbb2c789f97f74b175ed913f5d03b162f878', '461d01dcea8af2863ca216e9b9379c79870ac44ef18965e5263e0dd983dcc12e', '4e6612138b25de582ba274ca26709085719c519d9732264385c3a415a4be1396', '84193fb5acb01ee751fb887cec4b3600ee2037a8761feeddab288a623e9ddc69', '21e35aebae69610a739ce7c7a63ea3f44d833062f2592f875d7a87a30cda51f5', '5e1fbc5463d97194797b03182a50fe1553de60def7c077c158b1be348eafae18', '8a3b2fdaef4ce342bcbb0af1001c911bbcb6e85b72749a0c9c7e2794079fbd7c', 'f4356371f8838922d2d48b0210f504440307f6fb5356a92026f65970c274c1d2', 'd8f359613369ab9e68e6863ae0047488ba38578dae3b20dcfbe296077fb46595', '88ba5325dded037b5a39b88f4ea1f6e690873af270bac205f9b3b1d9aa499bc2']", 'layer_second_eval:669276030', "path:['c0e57d3f239fe31465ce24776659b7567354dcb7768a335309b0218665585d40', '2b746604e129ad4f1ad3e2c03115b04ad9325d8c13866afdb86dd13067b6bf23', 'd05a703d1b994dbcf326b660f9d323036b8aed391110695e2339f7faaf9d67fd', '7ae190e4f7a7d9763408600101bab0d02364286ad8cad178436ca709654b60fa', 'a63e13b0861ecadb8d2f49e99a6999d5a5d03d78d1aeb061092cd64524f571e9', '327c6bfabb48539844d998e79c0853304be7584e1e38e680529a02945e8d9f34', '899779253432b63da7c1bff8bc00fc34f8e8fd5a4d27bdae801e7c6822cddb3f', 'a4575f96eced380840dba44c6e7a482927a7d7e8ab4d9e29e01a21867c358fff', '93deda4314ede801247af4d5560166fe1105b63213c117b62c3c559763fbf5c7', 'cb9c889bbf44576d929554c402b200c6dc307a441fe4dea2db8dd4e56c23295a', '1c562e827bde468f6fdfa4a2edfafee6da396b7eea28a59aa5f4458fbd6b541c', '66da585065b674938d746b3c0bbe0c3eeddd792c58a1169430b1cd3f048f3d61']", 'layer_first_eval:3149983932', "path:['a3fb4d556b900326f4bec217bfc82680782be728fb63826f929cd9f745c2bb90', '8a0be78fcb3068dd03aadf463c46a03978f3ba86b23a14f229b4ee7d0d3377c4', '7c745e62cee05e4b225331ca5b12e75548c9f35a5d23e513d8f30465d59205b6', '5521277d41f614de9ea617ce8876887d2717405a6d992ff099658232d1da274e', 'db7b91575bf39a9fd1494f1cbf9538e0f94683795e3765b2678f1a4e5e09f630', '4ca392818e12ca526f9b280ddbdc545f7d472baed531757a38cb7ed9cdf6bc45', '55f55c3d8acea458358ff963a01e10c20e54a5c4b24e900a5efa7af627c5d6d4', '12a6afc86b10dd27577da415a6e6530500e25656de8025297203bceaddfe7280', '571917b72541324fd3ef92a9da9bee382cdc1297d7dcfd7b87b0f862a7f81cb0', 'a16821d25200a61c968901b0501116ad27255b87470156ebff5915a11b1f0785', '00c3a9d3314001ae5bae8b0432f865466a69a5496cc56700d7ae9ae66490e5f3']", 'layer_second_eval:1785294607', "path:['a6fb322259a97a3c37f1397d4402d257e9ff922c007327318cfb7f5b1b187a97', '916b3149b256f8255a55c5475d666daaf544e878768838e655e8bced6d61b9a6', '4e283a3a984d0314fe41d986b5b839db738751c04d509858dc5f500b8078010a', 'b6272584201f3240c7912d9c607ffd1a3cc2b2c174b4edaadaf5d35e4eaf712d', '4a54641d8be7716adb2204e6f42f0ab2536a1960fab48516f10d081555845ea9', '50a2ea5a6ad7148c63a40d45ee812b39dc58004dbd97d6d32ab8577b9ccc3d5d', 'f7d5af596d6fbea477378c4f3ef49465764b345995b9e28748250972e17bef65', '3fa75630e9bbfc570f4b2bc5939aeca459da49975c2ba157ba21748cb8a4f6fb', 'eaad8c3c1d3c758f8bd11d2bae4a693606b4970032d3cc64ebcc0a9d455d59f5', '00b7cbecebb6b31274e5c2b90836fa17fba9a7babbe4de87f36b970a25a0669e', '3b70c294fbe2f23a77cd733527d9b3a6980333d606e51b94395fa531f6f44b87']", 'layer_first_eval:3203803338', "path:['46fca0dd88e14e19fd8dd7ecee7241c404a36a917531c534decb97c5e7fbe9d3', 'cd806d97740376cdcdb2ea3bc1a6039c5efed085a6d4bd93a26c7a6c3353fc25', 'a2d928da1c1df9539ae5b8933677c5b0930f6a33f9e2ba18f318a4d5bf9d5777', 'c6390cb52e38b18e13797039594e53965a90d7b81c38305a29ff0ae355d18fb8', '2e6dd0f0327ae0a8db2a504ce0a00b59a4dba059167ffdc928c3a1e477e395c0', '480489976adb27fee5e86bb6c82f6077c261a8f1d071c732402db85c73406d4d', 'b4b0d4eba6c72d9568247aa464ec641bca7d4408f89ff1f5ef72329bbf06bb3c', '3473663f6bd50b4a208aa694041dc8b314c321646a9af7643e2da2d9475a12ee', 'fa97e6df0b39fdb6836754fe6e5dff3a01a3650c2d9771b7ddfa28ab66d51f97', 'b9ac9d9a1cf5b1f6e5ce7ffe5a656ae9ab69461c233e1ff70f5da6687a73c8bf']", 'layer_second_eval:706560475', "path:['70311be0576b3b660c6e3206a29bfa302778d9092413785e52445d696b169bf8', '65590840f5abbce4e4e00d31ce07b0054aa7aee6e8ce935a8ddeb67b1fd60493', '1a595babdc5ae165918b69004ca5dcee93bc2aadc0e428930eefa2d0ec29b846', '1bcf32d08fedf70ab6100c97bbef6e52a4b54d43ecc269c5742dd0da43744fd1', 'be681d874d78797b25dea5b54360fe77a692ed1f39824df373505cc51091d3db', '1fad76f4b59d8eb396f2d3e869c9297f9364d0b9c9702ab45760b451c0be0ffa', '2fbf1c4150984544ff9adc293a2f45181a8f072c816512886c2a3e79b794d65b', '3b99299468cbf8cdfe7bd4e46b576d4fc780c2a2b42e604ff78749763480506e', 'b22a4bc930da360f0a6e017d23588402d263bc40d679988b163e9b70efb2b8ad', 'e0c24f295f1decbcc590ef96c8d799de25b3e0d79fb4376cdeccac7619d5ebdf']", 'layer_first_eval:3208766032', "path:['2e56d9ad88132f648c44858d6a2fd4e8b1fe7a28e7477f67c4908a6768a2532d', '61f6540eab653480bd89bf240589c568ad554f9848a025188d7c1fa47a3496a4', 'c3ef614d5fdcf649de78a5ba8e2afc476e01d747735e21be959a23a772052e3c', '3841a896a28be953b563843b40007494bdfcf965708d4e14aaac34b98bec142a', 'd528ecc765a344950834bc2f9d93c1d01e1105266cd7d5456a5c9a6bbe6ff583', '701b82233316f0019f99795734a01150f661249af0873c535538ccf330473386', 'df1939e08d233a9f1d83e9e234d25399d2849f24ac63a6278a99f909d2a6695e', 'f005e809f6a11bb2daff3f560b747d0a3d5b58e2dbb3ab09c97e0ca6a38a78b3', '90673752206547fc9d2d7fc4ce52d5a6279e2146f5278894f1646458409d7c16']", 'layer_second_eval:829995476', "path:['5dce62b36fe74619bb6ebdc8f86e871f2462e7ec51ce8eb9359714625705f572', '50e37e2079c14c934ca6b725f9b4c374202ba2656b08fc9f49113514309852fa', '73838697d7b057113b358021a419a7b033703c50452452e2ba3645f99df7c615', '54800388eec6c1c3a77f966f110acb074740259062712e244f3bd06cd713c66e', '6b5dc4ea2db057b1bdb71fcee8985930850a67594477587a6e6d4344af174a50', '9a558eac6de3de29c206792e4b52f2a9b8c4046327ecbad12fb56abe6c32326b', '57bcd55b2f07fcf07f2f6f0048612a376be2cd0c838b2a5f4631290e046d60f0', '2c7f39b9a2907601ac3653d813fbaec67c66939b6d45a0e77adac2351c4937ef', 'db4ddc508fd32e5cb43d0daf4b1f8c2c68727bfdb6a1844d04c17055cb3f206d']", 'layer_first_eval:2163150272', "path:['d738c232745914066eca1d7398fbe3d1bf391222ef7fd5354970d501891f1f63', '656b7b493e94db033c911ea3cd64f0d8914c0c65f92269e8104d13864406c3a2', 'e03298b18ebcb20cca9aacc656ed1a5939ccf66baade7275f394833c554122fd', '7b1c6eba4f3d7941557916f494cbc4b740ac09cbffc21e118e35d2ab41918707', '9c829620631d7d3e1d28d08c5bcb2af0da53f9f30ddccffcc53a640e2bd5cfe5', '50aba2b48d117154d7ff465664b645267474036efd476686fc4742f29d253dae', 'a30574486d9b5f935e1f54487993380c0692e021fa73634774059cb2baaadb53', '895b8447d2411ab0b6108c314a5e8429f3a5d9d6739684f20aa0c9598a0d5248']", 'layer_second_eval:2230667542', "path:['6f9989032ba1485b46cdd01119876b6a6d0824bdcc99733c92846d85e268411b', '9844d4e3cb7ecd6709f5f18dd6c48c730d6b87556b316ef123ed894630e081ac', 'bbbd7ea1f7d3245f881f57f30c68210530943cae971ebc12fb79fc6a5828b9ca', 'f8f6e34a0772a6dce7c8a953e23d033bc72c4d6d70d1075fb4180704ef1c0546', '45397b8644ba6641ff73caa814accc010d748cdd9fcde6409075f33debcab71f', 'f1f6553a826c13aa1c361da48d60707edaf7dd171dea45fe7d134d476e952158', '1111cdf72e3f73fb354bcc28d8a698ed765631b7b9f11be719e43ed2a82a0bb9', '62bb5868d32aca6527297cf56e106317693772aca892dd6c566e51ba81379765']", 'layer_first_eval:1817119372', "path:['9dccf37950a199bb9ceec6449f348bf921905fd7d8f3df9427eaa9dc37805e1f', 'eafc4bd26c0325699b14f5fb7ce48045d4b57acb310f9535a74bd4fdc287a5d2', 'e3dd9b2cdc2f6519400d516f8ebe1d6cd1bea74ee8d27eeb8d776d006945a8fb', '8cc4cd7a0a6fe63d197c55fb2218c653f707b0a333ab41324c28c94db546676b', '645ec89001738d3dd15fb26e665abe197a6baaa2364273aaf68445308ebae947', '90ba6c3cfdc96f95c8dbcbc8388c179114c9fa90dcecaed97babddab0ed20721', 'dbe7bf2266cd93500c816550822bafd7a61bd02116e5fc071357235290120e17']", 'layer_second_eval:2250068882', "path:['0575bff3c4a8d8bd080375959ca2fc5c01b8def42ff452e7dc2208e6565d78d6', '08b5c1e0a7e4d733b3ad6bbc9f87751f2489dd2ddad6fdf8d6b3b2b47e9c9220', 'a1f23063149cd71d743ddf11c8cfb2dd3d164eae718286b9fedb1ecf8306c801', '5a6944e78b4fb24e0787fab182bb7b0a38eea7a569c7142c7bc636989a067c7f', '72920201cfa1ede5a803aff93f41650c6c90548bc7c4e956d72970202c15ed47', '05fc142ad4c3f7782ff960bc240ed8d6f8295e661c78679c008ba7e543400884', 'adf53ad0efd8e0154352dbaad08082b6555efa2916ed0a6497c9600559828bee']", 'layer_first_eval:2133197584', "path:['79992416e34ecd5647f2c1da633cbdc29f2279b23002a4e80c3d87534e7e1c49', 'f65b2e8cd9efd85b242a166db631fa7aa9b6a250694356ae58e279757c8be18a', 'e3a5a9468aa575cf5cc0c2fe39b76f8e0bed3a26d834a0f847dcb3f36390f4b5', '71510b0eff95ed89e2974b8a5646d9800bd643079848972245f5f46386de86e0', '863bdc42587a9d77b561e81df74a71a5cd4316453faee0f804713230eefd79dc', '5dd4fb8b5cb8979c59b8a9954905611d1ca3c57c3e016d36a58ce336f67fcb38']", 'layer_second_eval:1427595717', "path:['6b7c1a256ececd2985a96b75fa0502e35ce90cc625cfeb4ed0a2e4c81f80308e', '842fe5b88da4730cf84559c9a845685581fe56e064617626ad962a599802604e', 'e32e19120d823b22dac46ab440ec3cafb09846fdae11f0355c23853053e649df', '8179b101fde7fca22127cd5cd19073bc2d48d016d1b20ff09baa0c9e13209a48', '45d966b0d4e6b7f62d550035904d4c1e8249eae77be11c80d147fd57e2e3c985', '9f714e4006bba95ff256b236cc8223a68a252cac05804d6104cae39f489af257']", 'layer_first_eval:2049175622', "path:['c74b2f54b1fe4f238feec59856d8307004c8f3482cbfa4c890bee636193128ff', '8ae26f98cac14ea6dfc93975f4f3c62bbde155c610a66a599ef5854302507939', '1487fa0692e4e683a230755e20ac00af7b15343d486e319f3bb9c203a49e2427', '3bb0bb8fc1f5e9ad5b6a21520eb00da5842bb42b800157b5ba8181acc411334a', '31944a1267a8433a26573796dfdb394e7b2f950a984b6c3e4b1ae3b1f659b1dd']", 'layer_second_eval:1670558800', "path:['0646fea9e618547581637a3ce1ad14918cf2210d12df1a1dff655af72fa719b4', '54e1367b3feb9ac526cb1bea548ec6450f20f1c9af7962376e32c2c51c90eb58', '1351755569715c1bb24361b2be93a4a5e79a22fc6a787a59c35bf8a777ed436c', 'f391223eadc39ae28a8caf38853260f1a51ca9279d25f4b403fb62e4d79a9423', '52d149304485195c7353b09501b5cd3faa8c3ec5bc5895723d612071e2efa9a1']", 'layer_first_eval:409980660', "path:['ab57d53a25b16c066e5cb08fe35873350bd8bcf00acb3d830060f09d5d46637f', '9a51aa3a060c1f9c1a751410b8914f62143fc3879454c11d43121846fd96119f', '65fda71458157461c19a72f386de6998ff8a526fb1d9b800e970f5e206495a96', '67d9a7b60c4a92b3cb0818ae8dedb170566c5466eeb0214a8af8f50c7529b4d4']", 'layer_second_eval:3069539465', "path:['dd68a732de5c2cdf8e72e8e3fcbf88302d4fa53be42112c036e27233afba499a', '8b79dcd4361458f06e63226177792e875baf508f0be24e10a86c144d5d44ef6a', 'c3d4eb42eeed84d0f35b44cd95b4cae9e7ebdfbe4a0a856073436f310b2a9bba', 'bf03596210d2a581f6b744c1fbf7e893bce2785d4c6c64989a59c13fe6fd973e']", 'trace_eval:1128935082', "path:['5d20a8424b87fa411c7dc5b3e74e8c9037a1797e25eb04598a4ac335454fb2b1', 'ead7ad4a753d593c485bc0c3f4e64f2eebbeda5d65e92f9f6887a37f28aa1233', 'e943924f53120cfc3f1aa89ed3bc847faf54fa62adbc28fb7d9f4c7278e13cef', 'daa9fcaa6bd87c028ea40fd8403e3b77bf724dfbc7d5d98fe80ce40d6619b151', 'bd9f677616c85017bc2eb5502bb1aa919b1ed8fe25d379d22fb09cab8a675f11', 'ba1c29e3010e86d0064844e7500553c5885568e632bb04919d853856248c4dfc', 'f0f1bdae09a2c05611bd992aad9154f211aacf73b8a09498ddcd3959996cfa35', 'd1c0ac6725f8dc7561451db753d0c1b94fb84530b7a138245ee650c6e8f2d5e9', '8a52e8a4f33d1e78ea061953103c0a7aac7b9bef018a62a787eb35a11a7945b1', '386394b32f9b71986bc01157049e77e530daaec6a3725b8a6a1aa80416fc4810', 'd21418e52e7ed57598cb8108b662157a827498c2978e50d00998e4ff58539930', 'c23d3979d2f5cfc0963333fbb81a2ceab494ff93546dabad882a5070be1189ae', '9c1406a60184803a7db2683a0e8b0eeead7c37d9f16672e1c9bb5474ad0b50a3']", 'cp_eval:726284586', "path:['e659632634c5c589bc765291a081f47c10f4e53bcd71389ba2d82f4a7333eac2', '41b026b0bcf030269aa482923bf07ae3e283064ef38b22edc3938e01e1e36b40', '23228486694804446c978c96229eab84edfebb98ad09b657011e65aca7508311', '49e6ff40c21a6ce7fdc83cfb87e6d3c526e28b68d15207f2be43c762fd1321ec', '494ba3a86ebbb622e23303fd2973f2db53e8c6514db8b2eb8285b0e03c9fb65f', '63aab8f99375213db4dd146d1c340277dd91a6e2fa976066cc86ef26e9cadb01', '81b39454177dadcf851d6511479210849e38480c84ca0205ade23729df4c8235', '74ebab0db06fac9a15ff1ed40876c091ec0c5fd7f2fe3ee857fc11a35b09d971', '017277543b6e00302188439d5dd557fd1575d150f4744e13000b2e38b660cd32', '6e1c477c1f5a3e864ba3cff08e4d820019d457e83b63e8eb587e5a627e76264a', '2f28ee3754815136cf1e05643a1322e9f5a5ddd5797ed0de79ed6ccc893757d6', '6d684cd0436fb650e5d492f0e44526981a91d749afca7f0ca796e731c215c160', '9e6ef557912adeddf808784cdd7a8af68f8355f20d38a3e7c5b8f9ea37ef48b5']", 'layer_first_eval:1773249842', "path:['47df4d08741df081599274429591e83cae8405c9d075e3bec085c1fd0062901e', 'd6179338e9188cd0294572ad0f2bce546e3f126ed2c4d2b501e525c5ec54df09', '6367a2ac5b6eae579a67df207b264e5f5dde740a4320aec7dff6fd0afc994722', 'eb04a94da7f1c7d99e0c2bbca77c00414e0b7c56d214a86c21140ca8d15f3d4f', 'c7783378ec0f7d52ea058e2a910dec8f5e0056a589903abd7218c31f4a59fcef', '394ab2b41599e0f9b22282d2c13c3b2364867afc647ab809e5bb6fafb88365a1', 'f9a85b3b6eb96ef15118d8784e722e21e63bcf374c97064031e7315481c0139e', 'b0f96fc6068006be3400f37330cb46d3bbf5285049552d82f66891f0c9f9910c', '5454bfb9546931f74eedb41e9812c8333b45e646fecf4d21f1e269455d94a753', 'b108bca90afca972eb03586dfa029e0ec1c03ce3ed72ef7e4946e0dcd8e41766', 'fd2037c1adf88fc5e3d53c03147d63a8e9056cf96add5d65e9fbfdfaff944aec', 'f32cbdc45b4a0e5716eaae9289c99548b4ef0c8d57291d7c5c36f08ffee557a1', 'f91020be7cb8282ff9226196ef53e97d8fe4ba07e71e79c47d2380e607932e35']", 'layer_second_eval:3158810009', "path:['62e18a9c2d1de2d318f5c282f1f0f7c6a003394aa7209c16991695128e187b3d', '4c4dd9c15dfa7cce5a6c60ef24ebf5c4fce8ede6bec0bbd30a48441df2fba298', 'ac5ae41358b25f6559f5afad0bb1972bd4e016987dca1d837d305b1e9090b4d2', '44e9239eadbd7b92417fcfb826a69d52dd2ee4a97aba09f23dfaabd6c3514c8b', 'e3c2b44025a42942b40ea1547495db20827e61019f07a8c8c4cc44b055fcb0c5', '12e7c2fc663829f98ae804fc6e3c154591c412d1dfada8f93be57abd7630666f', '873f30d840adc67f2a579250ee13685a246aa2c7ad5dd9ab7f808cecc46d1c7f', '0fb91e2f9fabeb7586f2c05d7040c2d8d1a06b86c464085b74da07689757f5b9', 'b5eb358176de703674e90a41c65918dd300abd914de37279819d830cc569e25b', 'efff5bc9cfe76c64df4b4bb08b04584b81ca9ef535c4f0e3fdbea6cf2c49045a', 'c977bb52d65daa3f0f5ad999bc78e6e23dfcdede734ee69ba59cbefe464b8cf6', 'acfa8f7d0bba8180654fef0fa5b5dafd1292c169cf147f87136b01841e96164b', 'a07ea554d0cdf7c9725322a6f73183bfc8b4fead29197cf6ebb1453ee54f9e72']", 'layer_first_eval:52251439', "path:['47e02ee7ff46f8a6e20cc9516e2a6d664e699f4be61c2f746b9acadfd078ee95', '9bca217b7e3718f9b5a8faa3f14b4f1f290ec54f466181e072d153ea5d7c9900', 'a58bc67cc1ae671af5080e6deebecc5b91e86c7ae780084db5a1b2427fb1694d', '437a3aa956c0d028167f5a593ebcf871ec2e6973a132dadda970a1ec5fb09db8', '9431cf1b5b13120ea12e04087e15a46891d84c016b55ba4ead8db1895217bd07', 'f8545c9ab676a73e5d1df998d020b41c4e996e866b8be363b3e0743532da1042', '6cefba8bc84e94ee5913e132495cd93bab3e988e511b5028cffb3066fbc6d5c5', '02ca88bab9f5a75c7139c66ccbd919fa5df00f1d455fb1f361a5d12849a22893', 'd443cbd4e6081e28e9a929f6ae5bdfae49932a69981e362d093627d9e10c737e', '39c9b91ba0b37a8c2ab93858d9a16acfc552543c16ceed19247c725a046a7b20', '3325c18d3dbc658ade8eefc27fbb7a5dd2ca781c236801ee96bb92c075f0d5f7', '66da585065b674938d746b3c0bbe0c3eeddd792c58a1169430b1cd3f048f3d61']", 'layer_second_eval:2557784902', "path:['84a71ccf59f19652a30a0cda47756027e75f79cd6a69f73ef57f67f0839ae009', '49942ad26ecbeb3eba6c676e93a2a1ef4d69889ab134ecdfa70085ff978cd331', '28466e27f1e305f665f04aa491ae64c0328d08492fe31297dfd46a18233d965b', 'a5e9cf9bdf06a3e83fff5093e55bc87e5c9c3587f35c80bbecc89113b5007fcc', '14f08cf3e63e74fa0ce86197fa0512a350b6634288a87e40d14d56a070683e76', '589dc2916c335347c458af865124a707643057b01295263176eeecd09ab80711', '481a36be38f87534ddca9cde3a16f85267c00629e9c677759ce1e623806be6d0', 'dd770d929d543cc83aee7a60f42af1afc52e5b2388ef3338696201d44ce1638d', 'b79daedf9d1f6aad86ff9d610f31ecce455503b698a0f716f3e0886cfddf53f8', '982fef088d6537f689c249709bf6aab77c3520636524825846a9accaa06b588a', '4647a5eff6cf1f3eec34b65d52551bcf56bc0b6b9656b65e18464e6476c294e3', '88ba5325dded037b5a39b88f4ea1f6e690873af270bac205f9b3b1d9aa499bc2']", 'layer_first_eval:2345443894', "path:['dc9cc0c71c9d6ad5ed7754d79d1354dc4ddbc55087b5b0e89bae203b34326f18', '0f3f25c48527861ca38402b5a2716440cd2f5adad0b90e69e1f8173d1b3546a0', '528787280d2f21a3deb16f64a6c94e5a80870fbe47e45f572faee1a755724b8b', '977a58e75bafaebc7b79f394be56bc191fc80e3c8c2bfa68b2a5da478883c027', '9c127032e30234257a2bd15be3b5bfb7ff24bf2e2cdbed9597101e1a31432261', '472cb7a18dcb6ef417a73a1437c27ff5e3097e9c4e0c172281c2c1c14c666bab', 'b62041c80c2ac01b9dcd9951e6361d46d735ea55f4fed9bcf97a2a305660d27d', 'c373df0ed880ac7c36a443f68576539ec622d4ae5ff5aa3f4028a18450b04e95', '8bd51595d7e6632654b9cdc64127dd7b5c8d01b92a3a290f6813104247315c4b', '6934e8c24cdb0be84eb6ba63eef20ab25d11247ecce0b65315233c19f3170afa', '3b70c294fbe2f23a77cd733527d9b3a6980333d606e51b94395fa531f6f44b87']", 'layer_second_eval:895452653', "path:['34af9119e58de903cf33770a00e463e5cae4505cf3c5b521e44a5966d03adf3c', '5298cc8bd0a649b13b23c66b23466e9e2ade4f36ee8b93cc243e6916102f1d0e', 'f097dafc914c753fe8e66a4974fe441575c48b5ee12f5a1589b18c4a45cff265', '34dfbcb3a926779045e917dab059b2cc03374ade2bd949ab4dd27970fb7a9ab4', '2134aa44dc8e004ca139ca3ff864a4995e34b4c5f9ea61a146cda8f786c25804', '41cddcf0a68f81f35ea7ddef5a7eab0f438707a24607655a7737e23e2d129401', '7bacd1f852fcce2311209ae924db5c461be1030519b64b041eceb34e7c45b51f', '71daa6120b0bce66d3761a24b49e7fadce26a0aa41723d37660f8c2c5793dac1', 'c7e267e3aaf03d1cfacba512c1c198805849d59dd506904f8ebc7b416b6d396a', 'dede8181379d92d3a804ffa855797062b9ac56e04258acb98e10447afda1a9ae', '00c3a9d3314001ae5bae8b0432f865466a69a5496cc56700d7ae9ae66490e5f3']", 'layer_first_eval:2405094546', "path:['a7be8b0b644b3fc4c45d8ec3e0ec031a87964dbba9704b602d3eb1cd52e92055', 'ff8a75367633f7fcca2b7a68db20c2f538cc0eb967cb9ebec5b77b3586229341', 'eb6df9424930eb36f95373680ebf40bd3ee7e8d82148e04dd16d1c860da701f3', 'e9cb8bfee9ef348e0e64a2262d009051d08f3ea270c4e5af1240360d4b277b45', '1434f113d20c53f584f2aa2c9194ec11e68949b84c4212666ebf76a3650eff37', 'f46d98df8dc58ff2edacd194950ffdad530af809b90dec6f41f1ec514a6f7aa6', 'eecbf631ef7aa836e1a88563406f34db37c74a74feab3bc2e1b27de504d6680d', 'c8c98141a4e7997a62f386470a8bc025454406f416748ef4898e01e801d490b5', '9dbb40877c2ba0c93002611d87f95e6a18079e57cfb97ef4f33f5166e9d4b076', 'e0c24f295f1decbcc590ef96c8d799de25b3e0d79fb4376cdeccac7619d5ebdf']", 'layer_second_eval:2285513272', "path:['e24a5e59022fdc2dd1ae70f82e7fa641c6c86af74eeaab0b45f2283d156c8202', 'bef8710ed4bfafb6f2caba4eebd4a57b6b774453ee8c56a05cc2e2ba78bf96fc', '59e93f3c213f503bbbb93d683c454498eec77c0713701963766b67d4d3ab1283', '0842e89d55e4d3cbb6477a84a24a621fba30c6e674216fcd0a859ded625ea5cb', '659d34170dfe5362218d043c1e061d1e11fc0c057ebf5888657aa410857d97d6', 'ebc44073b6410878f25042f33cb2e265669b3ad9a4aca8c337ea1dd5e0e67dd7', 'b9388593b0cc4b1bfa520c402a7ca74dd6fd5cec17e629cc6804de70235f12ac', 'b8e496bb7660458e1d8a124c43af96f12018e3a7e839a674363fd04202a4b9c3', '32e6cf040376985f958cbdc9f418f0486cdc6de193a83b08ab81453f9c3569fd', 'b9ac9d9a1cf5b1f6e5ce7ffe5a656ae9ab69461c233e1ff70f5da6687a73c8bf']", 'layer_first_eval:1125245962', "path:['e7e30e577a7b46c216edebddd2cdcf7f2ad70cfb782dab31f8218beb08883226', 'e40f442409983d27c3aa920bc4a73a028ec8aacd2d012fe6bf8b7054c05f00c7', 'd2a197f3a1880881034a77aa973ed3eb988550775dd371dd9c22e9f345b39857', 'd69dcd78e0184682f3235b519bf2e1dc8459f94f5f2ee6e693c5731b6f273c65', 'd1e0ebd0cf04fba2bd83ed7723c8fe71c4ecadf181546c5936479343310c4082', 'cda357b49c8e0c9cc83010b323781b9d552d71e4ac2178fe58272a5b3b08305c', 'e2652e5706bb604bc47ee3c7ad0cece5b849d232e947e75ec936ff7bc9c7a2db', '2c7f39b9a2907601ac3653d813fbaec67c66939b6d45a0e77adac2351c4937ef', 'db4ddc508fd32e5cb43d0daf4b1f8c2c68727bfdb6a1844d04c17055cb3f206d']", 'layer_second_eval:1905624470', "path:['14985a42938502abc4ced948c89bade466d83f8f5b2f422c89d12001112b3c52', '76268fb72fe058e6a286a391ba92005e7dba1791864167779e21c4f559519099', '6dc98c4331333facef3507ac523882538135998afc66f243307d6d9bdc7c9026', '45e0ac6d44fb35981db83c70cf7b2a7fa400a845cbdd6bedd2a749c534d7a648', '7b33f41d8defd3b31161f79a867a9d8f5a48c49e42a8afd4dcf6025f86e36733', 'e6b2bd8bcba04be662a16ea5ae56d735ad5595be287e0bcc8993b36bd7e0e17a', '0e256067f9d4cd25e3be3c8990e1edcf954d5780c56ec949bb78ac1b9fbe3c99', 'f005e809f6a11bb2daff3f560b747d0a3d5b58e2dbb3ab09c97e0ca6a38a78b3', '90673752206547fc9d2d7fc4ce52d5a6279e2146f5278894f1646458409d7c16']", 'layer_first_eval:1387863412', "path:['62b4a2bf20ace9a45c9136c98b516620428d1e7adcd8df23e6f0a0fcc62c884e', 'fa31ca5a406c33cfcc6eaa9d5a9015347c55dbaaa5ccd68e76d8bd2607bd5154', '8dd7cf633affd6cbb3d438553923f855df235491e1d6368ecaca265d20596245', 'd470750496a4aac9b5b1ad9069ad18eea3745a447207ae51551f472c5eb0e2ce', '143ff90cf7d3ca9ccbf3e60fc33409a2a75cd92998016984090572bdd4036fff', '1de0a53f11e47741cc5583c3df5c27914fc466a7a60b2f2ae91cfa24c2492edc', '57211157b33cd3a56b2f486995ae0ca2cc2b0fc5123290dda8cd642de30a8806', '895b8447d2411ab0b6108c314a5e8429f3a5d9d6739684f20aa0c9598a0d5248']", 'layer_second_eval:455024314', "path:['af779c9d16ed64e0ace8ee8abba62b30ff2fef8b81134d96f292aba9a2ce2cda', '4b00906f4cc1025a0f81ea1a80bc06d9faeefbbdc8ac9a0ab75d9df9447bf32d', 'eb1eac4d56f71a9d3e8af3b8e1e2563c15673028cc6f4bc45097af6c0991d671', '78fe757cf7388ea52ad966c1e3037709cedd5000cec1677d4cbddae052a23ecc', 'ced013c98e15732469834c3c9cf1bfad3d65e5ea659feab3f3ec73a4705504ec', 'c79b88677738ac99ccf940a00537839ec1b78b30a6cf513832dc935efc4f20f1', '566faed41e4744e2db9ccf098f6efe40e49c91c7b487fe98e9e8dedec9be5eed', '62bb5868d32aca6527297cf56e106317693772aca892dd6c566e51ba81379765']", 'layer_first_eval:1743796659', "path:['59e3b1a9c507d8ecebe55f60c4d563cc43cbf6fa478d1eb3fda88d1e2118cba0', '6fdafd4f6057245bc902d35fa014a9fe9623434d5b1203d5cf5552f265513afb', 'd6e1e3399e82a9935f66d7354f1ace85f81699e0d10ac88cf744863350972d39', 'd02dbbfabc436b2801929992193f1ee49a5a8067116851570bc45529d74e7555', '0273737a6607265609fe015d5ec6fcfb86964eeaafa8f456ec0e5b2f191849eb', '8c6db7eebdd7efdd13a7ba8e9d75f17b1b3cb96b7723242b8959ba887aea7f4b', 'adf53ad0efd8e0154352dbaad08082b6555efa2916ed0a6497c9600559828bee']", 'layer_second_eval:959501431', "path:['df335f705eec9f380916fa009a6228be8be74034a251d1ec2bbf444f0a55fe22', '846fdba16f56574594c192cb7d133100b4660c76de8845aca44b3e9f33ab8def', 'c12a1377c2e50b5850cad9422b7f92df703c95f79d6a50a38413ef3c9facc23c', '6a8e6ecefbb1054ed812e303e07a6f8c0492d13f4f43a40f24afd54b9ac6b954', '781bf12c764e1b17880275da75093aa5db573bd4f3d331643693befe0d36437c', '0438021e63102f490056d46d2a6bfd9318309878ecf8fd7f08502fa18cae9aad', 'dbe7bf2266cd93500c816550822bafd7a61bd02116e5fc071357235290120e17']", 'layer_first_eval:1618171975', "path:['4631d4aaa15c3dd6270560e6c3ab5d6baf2d0be84bfe719002388b38d285ddc4', 'fd302d5e73477ceef4ac0c4ac588fd86e58b32fd1e6ac2a9c6b548f6689bc481', '205d9e6617866da52742cbebdda1e2776051f74a1727f8f1c62af404f2d38bd5', '40bf4386480034bff65e2dcdf6c204bbcc584e098940a3e18f03083f3b9725e7', '7fef49790807f1e607fde6037fff6debf7f10892f61509a7739e6dc6d473c72f', '9f714e4006bba95ff256b236cc8223a68a252cac05804d6104cae39f489af257']", 'layer_second_eval:960767484', "path:['76c9c5e0cbc009aa4e603800c762946c131d59124b6a6caf66f42b5ecdd179c5', '94bc19f3d3071842af136777b5d9e94e639b707fc76d74d155cda528de6767ed', '9f66056d4472f028d35990443d1b801ab6b2c7768073b89a6fa3d04334ae5bb8', '8132f4869c01b3744098a5d49de6f529b93fd3f3c0a8a781034f13247df7f0fe', '83fe320d39a22bc58d9056d76dba08a530848940a0fab095609c05a47c0bb789', '5dd4fb8b5cb8979c59b8a9954905611d1ca3c57c3e016d36a58ce336f67fcb38']", 'layer_first_eval:929868956', "path:['e08a96f48757fbb64323cb7d90f388f84461be89b406d6e84e3a8158e2c97a22', '484f60983e8ee8dc3d3ff182c027165cedeb53adaf8de937b8bfada27bdde916', 'd041eb6598c71a475155b2f97ea598260e5a7062b57913260ef2abee134cdf67', 'f391223eadc39ae28a8caf38853260f1a51ca9279d25f4b403fb62e4d79a9423', '52d149304485195c7353b09501b5cd3faa8c3ec5bc5895723d612071e2efa9a1']", 'layer_second_eval:2868169475', "path:['ad8d9ef7f8f2a9cdf35fc6cdf3779b64442d6de61d6c294e7fd947babdffad6c', '976ec3737e9c914336a5bfa68f5120ced986123c28e60b083b331a9b4b35a606', '55afaa73c1a902e244aa1862879cd4487ee3f542b6c12e696e23d7b5dd7b8336', '3bb0bb8fc1f5e9ad5b6a21520eb00da5842bb42b800157b5ba8181acc411334a', '31944a1267a8433a26573796dfdb394e7b2f950a984b6c3e4b1ae3b1f659b1dd']", 'layer_first_eval:1541146205', "path:['53de010279aded49bc33a36fc4afc7ba258f30b5efea6cf7a1f2513113f4e175', '26b78a1a336ff30fd6da7bd994db76df5fd22c11856976a32e9a35e9a959b375', '07b0cf9fef3f3a415cffadf1bebd676fd884cf9e5d90bb3d8db6946e8ebcdfd8', '67d9a7b60c4a92b3cb0818ae8dedb170566c5466eeb0214a8af8f50c7529b4d4']", 'layer_second_eval:1938373920', "path:['57756b3b8d6716b9f6e9ffe013599c8eb0eda256be7099ccee2da9bf9d880251', 'ec2024b48ccc9dad2275b3b7a5913e5118e740599503eaea2afdfe6871d28b89', '533a0f8d903a14a5c00b036aaa734e0464fe2ab6f86a4825ca2bc963553be57d', 'bf03596210d2a581f6b744c1fbf7e893bce2785d4c6c64989a59c13fe6fd973e']"]
wrong_proof = ['eval_merkle:d5dc13a6caf521a524b6118c37d0a649be8ea23abaa245525205c445053a535d', 'cp_eval_merkle:4a5e9d3db8ea5172e4b35e3bbb725921891ec2b1439a17a733e6c3ee2a22271b', 'cp_z:803344509', 'mask_z1:2517630409', 'mask_z2:1586513178', 'mask_z3:1057354326', 'oods_eval_merkle:39c887b297ba9f7533b26b00e1a621dd55289ff0422f34bb31bec881bf852d48', 'layer_root:2b80f8fe362e7ffedfe2640d79206bd6833812e56cddc3154f640e314962dd1b', 'layer_root:ed6af656b1c5c92c786b7caba6fca5b8364f7a800bcadb5d726b566f43ee5e99', 'layer_root:e8c8dd4a684732ca3bc01a9879375737c5d0e8b7d278e64d294f1f5ff455ffdf', 'layer_root:1873d5bcc8a5bfc7d773005ee7ceb3e7919fb0918fc4349a4e41cedc110e9ca5', 'layer_root:5460fc5a1c6c07e73c578d0ded822700e27b06b5ca03134c02f7ffeb413bca31', 'layer_root:744ac9210c8abe31ddf12876825631c3cc14c11fb065b4b929cc2379c979b782', 'layer_root:2434cae5134e8f2eac2bba4aa9dff1a36560aece5f158ac02d49eba30c2f43b9', 'layer_root:e3b5c9354051818e17dda1a9c02e07d91edcbe5715c7c8c77eb2e18469bc7d48', 'layer_root:c3b47df0b08b2521eb590c92059a054525621ac5a331b6fef36c93db6bd154d6', 'last_layer_const:1908596331', 'trace_eval:3145533157', "path:['7dee49b21e83940d49cf784c19f891dd0f0a80271d9dd6330fe78e8897bab9e1', 'ebf4a10bd768b2d0a56fac3183ad69a9af59771dbfbbb39156a635db894fb09a', 'b8df3c9833e6b4d3217d4cb441c91eac563d5fa30264a8eae85bebc4f5f17403', 'd6ce3ee98403bab287822462f3d221e07770bb807d669ccf1fa84316d4e340f5', '1b7a8d0bf0d123a13cf37ec2fcfa633372831bcf25cccaf3d891aaa8f2cf37f8', 'd2221ca7fc884b316a92747ebf028f9e983a85e8945106b797d5b99448f457b4', 'a3abd1020fd07d22e59e3fcf9b25169c5782e6c9a1c563a139c704e22cab50cd', '77e72ce6bae66955799ad05e1d6fce1a733ca90a0710723c5631bb333b71a926', '5f501047c23ed5392064ff50e0dc8c8bdf4f6061526fe94252a3614cb4e0c273', '58467a4f495667cfc4b0174b21a281be3cff97264a7f3d7ee14a8eb29852dbbe', 'ab9d54dabc4e1e44f8f9f68882d31e8867346f3230c40187c079080b397c02c3', '9e2834f9f07eb2e7dd5dd66505b525c1a8a1068f0f3c07514fe7fbe2125e72b3', '7d80d9356affbc62f83b60e218d3f277b26c8fbb33049d7e22787be98cad225a']", 'cp_eval:497002522', "path:['50a09d36570e9693c82b4c788a78bf1dcaad770b829448a0e0bcd3f713a9afef', 'ce65cf9421e7b88eebe051dcf94f3c6e7208db7f4d28ee20e2825e6c73600da4', 'f00891dba75c16e5f4fdd918cb5123cf47521945915103efea38c3b77565d4ac', '5a871c8344a6459aa02a3a5abde06fa29d724aac54cb478e41fffc0676e2f476', '9135e90a2930e90b06687eb5f5c6e68f10ff2ffd61c82973b48a71b574f8fda0', 'a7f0dee7cf4c6a1af46dfbc6b01a2dcddb9c73b376109d6db472b4d1377c5fcf', '6136b8d2f49392ef66dfd75953520c4f4c7f7902e98cb69f9156f6db12e901b8', '0aa837e0ed0620f962fda13991934eb5d2c33d730280119eb5892d3e75510d18', '16061054a50c03c343536822a04e6e6db9c5c7931f94a22d5c87a9116a59f84c', '944d42a5dfb7946b69e2e8053d4fbf2030361574f9870008328c2d09fb9f2cb2', '159459b7bf421a1cd1f7e1afcaff5561393a740cdac4d37f23be16f6e8e7cf90', '92d36c1753a7db5112a2bfdaf5d75a761240876251c019bed147019c2efc5c2f', 'c90db6835ad27b7b088b9de56aad2445378e6354952c43f92b46d30332c2ad4f']", 'layer_first_eval:2470041122', "path:['0b69a3f1cadc4e3b9841cab970f11c9e6d9acd95004c000bea8a3e8634059543', '2f77a7321d423ecfa7b673078d2cdebda4c9af3b9d6839eae976049a8722239d', '9d810a38da2f1fa97420c4a3b3415a5c91d8bdec50d2e9d6290ff1d784bbd2ad', 'eb670c2f945971f007fa0fa8170c4d6c944222383b70670931bf385d774fa991', '8f28d07c28702914f0b0b90276571d6df19d25ec610caca7b0949b6c1b4c095a', '7f6e161c44808d3fa0b692703deb37a97fd693d8eae6e4a283f040cf396b1fd1', '3ffff6e6e72f577e2163629b8def6fc91ab2e6e8239e956e743d0168f0f41be0', 'dc2c79bde9507b93f386819e9425f38a235f20029aa1a7621c33fd1840a61a7e', '947de0116ca64a445f1fc0553f8a65b4e8d18c61c8be7b6f14873c5908ef55ef', '22177df63d077cdb028a374d887237659f8a270ec290a85efad6443a5d38296b', '01cfe2484754c9cfa99a153cdea553fee4c45a84138f9f5865c479310612ea87', '8570be0f9fd6179317920a8e3e0e9dc6b9701a5fd5ee99356a30cbb18d4ba674', '66b44b71b2457bb09615591227be139cbf42e02a18782e4de7931680f16b1703']", 'layer_second_eval:1419507409', "path:['51e5c3b2c641e2b5cf61078a19512ba6642f1cca7679cc1707a3258ecef30f62', '31ccd6acdfd19df9bf6308399134601e827f2b9244a793bc56b025489150424a', '7978012514a12c17c93de24f9f93ee594c375e940da6e091a2c50a101afe22ee', '01c29daf4f03d5b27a981579bac9e95d994570b04c2d3e4f36b01dda889f3f9e', '2cd772cb561bff68d91992d3eacaf38718da3dbe893b6c4379e7a92e35a79a75', '5bd844eeff367c249e2ecf3828a1f62ff6226d15bc49498679c8a8c25472b41f', 'a12148bb7c6d535f01387a41999ce74174bd83297aa27e89658d70c3fb8a9799', '71a43fe53661caeed338c8b087994a69982a5440c40fd681a9f713bd17be7e06', '13da2c795072317972cf9ed5e660f67758f65dda1affaaed4dd223410b66005e', '60454e229d60b739022730db4cc2ed6b3c4e67434108d4d761db278453af1f6b', '78033f0783eb746f2552e03e290b69abcd59d43cdb6617e27163e3c4ce431e1e', '45ebb01c317fbef3ec86da652c16f8e31d968402a2b3e95fbe91e8502b541bb2', 'a4a4f05b38507befa036d8096351f9046896483b670cb4009777bb57398ddd00']", 'layer_first_eval:667241179', "path:['a58aa2564342d2960ac7d44b1e356aee7d8bbdbd66ce6eb475fae95217f014a5', '1a03362af6d5f25c40e7b1bb23b5f57baf36162466a5f60cdf0d14687c2412a5', '226d648592ff5bf4b1fb3fe93d4f865917620e35c0fb1a34a5f233a8e12e61a1', '1f3ec2130951734dba88900923c0bfc83a9425503341d9176d5e823bfee839ff', 'b6740dcb53a9f006afc74dffba1951fd709634e6cca133d664dc792045e5f50c', '0be65d22029a814bd31b36158baf86a8dfddce23046b2b5c2d800d857ad956a4', '8fa3b816f32ac4b5eed477f83d4f079286118da1093657fddd0050d80c350004', '6e550543918adfe0ba3e6d0d9910249bfcbecc57ea2a692b24e6d352f72040e8', 'ad86fca891f8e7cc9d856cf31bd36dcd8dcf72f6bc0d740ab0cad30e3a939b21', '784fd20215383957856f1e9282675645bb6c9f7cccd1528db131efe8ff2e2e68', '911dd3e2eca3a8f098f0bac4519f3ff2b09c7395089e2288eb7bac640b4f50b3', '0f310d66046d7c118cb0df430c6bf21f3641962b7501e5e5645d58d9bd201e3d']", 'layer_second_eval:3208140555', "path:['d6c1bb152059842141897d20904e2083ffe9e1571deb06b917dee1ba4710f6d8', '7d152b2a4b3302c6f1f1222c4698915975213ec02fddf651d300ecd03980041e', '87c85573bbc7e33af9466cd8922e32e4428e0a738b4ec2f4aa4a6d0f5f8da2bf', '2a419bbc59dcd592a6ae973ad2fafa43adc22ed2f38f99226aa84be8c5407ecc', '0b8e4618b4f6867cb57d7641baf289b0e6d5c3356a1a675d820edb2191c5d0cd', 'df1e410ad096befc9dcc829dfee9fe07cffc5c1e6af150f4ee7b0293e57c5e75', 'f8205909924046fe60906e4b53878cebaaf0a05627d6eee8958dddd01cb3c76c', 'e3277fee7b3658486fc6d6b3073c5b4212d6a6e8f2314f263917d190448ed3d8', '0e2fadc806c15b3eaa4f363ce179e5390cd1ea68d2976a1b019f2011658faa5f', 'e466d1de3eecae8aadd8143c095be7558f8819ee5ca3b3e9c07f1f7f48494ea3', 'c4f2b928fd3a83c688e844f656aa8dcb4b1a05b17b9a55a0fd5a1f5cc4a1817e', '75b0701167193125c48ceceae35f4c411c9aa5ad5fd345d6cf1c246461c6319c']", 'layer_first_eval:1246058955', "path:['ae351c6f01561d64c722b4cdc75d427553711e5d844275a95d0388100c2b5203', '604af0c29938b3fad32a107d6e14e0fe397014fa57e128c17cdcea543c302b25', 'c00783694a47182e4eb1244a413003a6b3fdf85e08a2a310a72d00c399f9d5f0', 'bb1d20d56089bd6784888f4c6a4a68ee4cdb0f5bc9ea79d4a0291c3107a93af2', 'cffa75a09d76848e02ba185dabe84e1249fb97647555e1779968e318db285c13', '58a50fd1d75e7d5bcfc3a5a2ec7f2256f7807220cd3573180fc42d3efe7379b1', '71b0a298b4bbc09e9356596fb37b97905209487f3d1360dc7c7d3911b4b378ea', 'd17a39ea991941c5a7ee4dc4c1c4a59ffe4d44bae6a6509ef06a7a862225154a', '0e9e00598c514bf65fde04afdafd879b36553a3a3f8637007184f9b6bd673c24', 'f376636e9530c1700aca9cddd5f757f1ea6fb03fe64b315716ddb6bee1fe3806', '53fb63219c80b5dbb697c4589028f1718f59c5408e2c940e9071438f46552412']", 'layer_second_eval:1632658190', "path:['6080b75da118a68a5fd143aeb5a1ade8a07d0a715aa9ca2af44ff743a366dd62', '8c10c45cc3e7e3cdab644bb10c8e061b1777e657a411c765fa2dfcdbfe0615b6', '786e7125901e191d928f078f2221a2d3b0c35fede628b0d663d70755dbbf580b', 'be9fcc89ed4750e4771a75304d9636c37022467901afa335e7f2406a3789ab8d', 'e69e1b7f3b1f661beb2ac8f325f5f11a8e00561b6880d4fa73348bfabc72f26d', '58893c4f599aaf0814b5669baca427c2e53f0c84cc4803bc5f23f885b311b595', '40bf887f6aa30414e0ed19e1ef623dc30cbc0dc24e64a10fb5426b7fb100c83f', '0093502c2d40fa49ed0f438e62ca09c4f7d0ff95e0c65a4820a1d9f63cce356e', 'c43037b17c5b0494e9e0fb4a5411eeeb262923a4ec1022866ee2c338b1042555', '289f4276e5ec9b265957888bcad6fe286149aaf57f3ed7342ce15ce03a8004d0', '12a5a4aea15883e396cb57e3fbe55fb1c3963758c3f05dfe5b4bc6a3e5ab13ee']", 'layer_first_eval:1010712822', "path:['594736c64e6eebbc573a8e295a2993ce906cec432ce73ceabfdd57363dfb1bab', 'ed7d061eee1f24df935d518e30043038fbd9df0d81fce6b44263b466e5e5a172', '07baca989f86d0a2b150718e0a224ac7932342d8c3cdbca8f6d41382b41ee6b7', '98365962835e946fa7eac2ba05efd6d50459b63bbae87ad88e8636f272e80959', 'bc7d0a9406481a0ef15546d8788589999383e75075d278c362b8d336695e63f6', 'ae825229fad38c3b0db250dcd5b245a948faa5754966cd3119687c9bdbeace2f', '846e039bd04f8a995ba9a75985800754c844992dd05a50908d1936bc623fd477', 'c798a87fc7588d2064729d2ba9062e25f73f8768bc2fc61ef1abf91dcd57601b', '466614e004a849117aabb18428799a552223b1442f818c7c75a15794280c7375', 'b8ca5c278a07b3640e3730d0403cc58301eb50c7b8cc6392a4117f3b0e25b23e']", 'layer_second_eval:2629070779', "path:['e6071b7a49a775f87a49d79f3277acc061d67a0d35520b91631bd483b980eca8', '7a83bc7d552943185e578b14bbdaa99728016dfea9ae74a849c143d13844aba4', '45b546071c6c9d562a4ed863309539f84061eb0535e70a4f152f9e9421a6de35', 'd47d8fd24bc46f28ea780ec5c82e47387d525830f1a75c56214f4eebbc2235f4', '14e5840c09dd47a082a058816684f9a90a6c0819773a738cbc38b0b4b304487b', '638c28ab30b4f1bb0605686ae80a3d1ed9c0c7e3bbebe56a5619c477507cee51', '31001ce623ec9395986dbc2d8465393fc9df6ecdc646a5b7b5d3ee2eb99620af', '0ec49766cb78e7657836b799f5ea0be43f5c4889daf026e5eabdaf5654058de0', '86635364d4e0b6ea25128f8a6db6c9950944605ac2faa33c59eb12eaef07553e', '7dc45620dac8c491442099c6e4d812ee363fc0e90bbd83a12d4945b202f432ea']", 'layer_first_eval:2752442351', "path:['0d0cd33fdd16aef7d9a7c0c18087bb309d5bcb0e9724823029b9d7392f4311d4', '1f9be9ccaa417e99890668b0231fb3f0f7ea4983b28c77406aab38249f0d3cc1', '0cc645a5ceb7753ce028e6d54f7406f64ba3f258b8357849946e873f9118ae0e', '1733597df20ce402cb995602e66d39196999a9f3e7a1cf49e7f1c89d778a9c60', '43d309d0ef94ad4c1841c0a0203cc5350d4750c21dd61e4fc619024626ff42e6', '74a8fc487e8fd67678c43276ae1f61b95773aaa8c15b395fd1a03d3af735c92a', '58d8d5fc3273ae27d6c392a57dadfedf03f9d5ab05b5fd442a7b9dc9ee1fd2e9', 'dbc5231870b48a5085be42206fa309b03a118dc359170262eba669903a3cb0a3', 'cfc1202c6c87b02348ca6f07628d0aea012cda848301323904821745389cf6cd']", 'layer_second_eval:407364563', "path:['c47593bae4dce03a0155df3cbe7b4af47baccaabaf77fec00e58006addacdc3a', '3aca1f96d4de1e0ef384359b75d69bea9dc4d4a752bde488231d1a18d3f51f5b', '336c87495a384cf56505ad6db440fbbdef302291e8c965209dee7fd11ca6900d', '9125732d04939cd8754c3388346dd6eb6ee5f00feac8a70a23e69e22f63af3b4', '8c2e5b820daf5fca9a6953be38a5dd4412f75c37bcf33985340b91c589990d57', '3200a003d4158d8545bee71f1540c0e9f234c6eebe62111f37a3e0c7c7ce9f27', '37bf0dabc3bebfac64b43199df7ad93046d0407eac3d3bd18abe4605699a51c7', 'e659e1f4754bebadeef8427da781115762ea0a4f0ae74526c03a48bd3886ca28', '0910186530b067582f78578111db616664d7514d4fe86110c7ac9f60d58b5b90']", 'layer_first_eval:2681268917', "path:['bcb0f8516c6ec9664c19a37d5c13e419055498a9a9d6f398a8fbdd2da1608903', 'f3d865a12a1de33f849876ae87d4bce736666208c14b0932280a8e3521303646', 'ff021aff73a73ba16c49f6899082a3b99e4fcd82f15cdc76e84aa32c01f4e564', '1015cc2bc878a36856ad1c101ea165574fdbfba50c530891fd6871fe31ff4db3', '9c2d9b67733e3bfba470a662bd29b8c366dd2c369d6b437685aaa8a1f064449e', 'ca2fd972bcf0b631ae52feaba6221e122a1af51917481202df160b4d21a50e9e', '0fb53c7d125e69194228f3ce27469aa2fee266c1a6243882dd4cd37d9995e08c', '8ae42a962801794166bfedd43f51240447348db7b27ea30872dbb3967372f4ca']", 'layer_second_eval:3038096757', "path:['c9b4ab775507b176f4df105f5c0a2e1499bc22f7eb2918ff1a68a1c7d42c11bb', '29d88e00628f9a8632e135a5b6d22b7405ee4457966af80979678302c27f6408', 'df1d5f84775499843d56393c7ad1b6e368d16b5da68a5a3a6eb65d72b39a8277', '56e41cb278733e878bcd00868d3c7cbe9c138259ef3f0d2024e2aec503d4b047', 'f7e79a8d32e2bd5800d9f9dbccd79c66709c2745676214f2bedf587902635107', '331ee0e4901c8c9b7ad06318d1ab2a805bceb2bb831675ffa63ce85d597f0d4a', '89dab384f03128a33906648e4d188cf0fb59f0074a1eaf025314893712e14de8', '02e070169662eeef4764985ff525d891779958948474d26098f178a68ec32ae8']", 'layer_first_eval:80269777', "path:['70ff97771ec27c874c4a43fd6d42fa57e91287d818d16e021933582bb13c675c', 'f511dff0c6738b2b0dfa02f60e18efc966dc263126f394e92d33782b24512935', 'd1ddf3b56a25b85e3fb8d0bf49b9c1751c1401c127b4e72eadbd370c2bba7aa2', 'b656851e45e297f00a858b491136497f6ae76489b5ef0d8f0e714c327b2047ab', 'ff7e6ef7d14eeede3d95027b3aeaff1328edca32365d5a602cef07ccf2076d13', '6795e0ef4336aa07d99dfdff3591ee2839dfc8349bfaca193b209cc47f66f1ae', '69126f97e4d83d47fd4ec94fad9de2ef70edff0745d10fd22399937ef89e58db']", 'layer_second_eval:3133420302', "path:['2341a39b2ddac2c1f44123dc3bda6443841dd16eb865f0434c9689ee14d37ab0', 'e6b2ebef0b96ca2030f8101142fb89cbe0d42200c93e8da61f205e9be6cac734', '9b44794a0162c4299eb158b91f96acac96acad47d09c0de85990b9d601c9d03b', '00bb2aaeb37be02827d618960dac7d57bfdbfc115b6790de3136e6744464a865', '9971c1d1e2d3276665d2f105fe590aef2aab92e79f8b79fcf5835eaff8007f44', '479492672ed296c5f0ddced0a4fa2c26264772772d1c7c1bc55052e0c955c4de', 'cb7fa843377ae7575a2943eacd482c81ec5356a323f090b5b7c2d0789d77ba12']", 'layer_first_eval:2098476308', "path:['f625f209395e61e4f4c62d1d67bcde5cd806e7dcdbc6a4904d0493532eb02763', '91e4be5335e95b87d730ba973de3e5e49b5087af13d37bd50280cb03000bd77c', '242f0b8277b45cbc3247e8321864bfc0e92032d05e8f87dc46fa90e3f91d7814', '9b1bbc60d0e7bf4d78d61ace9788de9ba2907d2dcbf6d7a47f512fc59243aa34', '9ce5714b43360c3176e3704ba48488bafccc44db68451c080df73ef3bb76b503', 'b68d524eaf64bb35f5a56c6c68c357ef6204afe8856ea9821accbc158385b8ad']", 'layer_second_eval:469838335', "path:['78f3b8bb4d02853da51b6fc015834ace0e27031a5b8c4d789be11923f79507bc', '16df4448444624c5d197c58463ab6e9059b5b75ce4a7ee8742ca9a3a52d21292', '42a2a76eef6c9b6e3e4f27c739b7cff67bb947638b72f733828d2ff011fe1a65', '9919ae1545db55a4e5efb3beed35c53d99652bb5f2537657a8396e0c07457085', '0c92cc27ee8200d2451314ffe2801f98a5fff713e3067458fc4d9134de4ef47e', '6bd2c8259f7fd8a5586580d355001c869ccf1f42a2d835b100bc0e1bb8aadfaa']", 'layer_first_eval:307430880', "path:['be49c6381417720d970b43cfc4201272845b9fefbba2b9c8aef3268042aceb36', 'cfd05352e66ffb2ec92178af56461b8aa0c99b6ad49ecc24b94d24bf0bc0a92d', 'bc3bf844e7683c3eabd9bc1c7a694fe1e0c85fe8d262008583334fe6e867ddd1', '578df4fd6f41ea24a7b31d8da5dc6064874fef41a81179b36a0e96ab6b378891', 'e608b7ba7be420932014fba06c1d7184a336c76248fbc7192c52581bcf423c93']", 'layer_second_eval:84881064', "path:['560de423a63ceee5ad382c2585761d41a3d6cbb8f4e015b2e61168c688139934', '6f92517246ba3a0812a133c84b5acd0c6caabdba070e7343a7bdbc456a4f5627', 'a7d63602512b383f54b77c3968fc545942bbe157a3e673dceb9ad3f8cf548c37', 'f2096329e2a00f0434f7f8970257968e01b5e5ad132e5a0c4e64a4a1c5009f66', '140db42fdf3e2ec6c8c1e6b085330a7d54d8e0ed55f6bb35513fb9d5388a5f24']", 'layer_first_eval:2509621709', "path:['316db770044d27dbf2e7af7e31e886274fdbd3eb591d954cedebb00410e771e1', '83f551aae85628c86df0b4da79f6ecd7701747498c4f9bdc65054c264ed0f235', '43459233e43ca5625bc6a6580677cb8ff2e746f3edde656d59f9f39e2115c2a4', 'bcc6e5e80a7789698f7f3c1877f045bb2cc7aa652296a45419ce5bcdad3de2ba']", 'layer_second_eval:2917702132', "path:['171e932578639b4a6c4a4161433c04e007096d33dd54129a36468fb02d684f3e', '1c361b58b1693492241e7895efe5c410475eb9f65fafae0160c65e16f63e51fb', '27c6c37aa47a569dbb4804e91fc00e0d07d69adffd5ad30fab524d5628f6e23b', 'd5386d3561e46aa598c7d376322fd26eb260a1b6ddb75f4b2bdee8cb5ad3e5cc']", 'trace_eval:2163897792', "path:['e4084cf20854c15e856e5dca235f4eb516f6a833782f8e6301d4fa4ffac0f273', '5cea63a09ac231995bebc349635765e628135fbca575a5549cd536664c637729', '26aa7dd78fd3ce5a9cd74ff8db9e9f4da3ff97def87a8448c11db4f718c09480', 'aa5da4041391b2a79ce95704c2ce59615c4063f5e6d58ed7d4e8eaa100ab6f96', '9abd5e88a835876886b2e4c5d6e23a49b1c0250aac716c8a62099b67958e9c9f', '40c1053655310113d43c22b6f34316aa299efc4d6aec34b066ad9fe8694a6cbf', '54ade6c879b0abdd3fd426a754fb4aef685cfafd8b875996a649ca84db7d6955', '53c8f53d4f5e31dec076047f8c647bfb182d9f0d738744831f0920b83ed48611', 'ff7906d163865a3321534436468c38ee0d4d031861377f5fb9135713e8663f93', '5f966a840c000e3aa8c5789817077c92a09488ed1cbf3b168761a661e527d4b2', 'a5d875f072130b7fb2aeead67aae26eddf0ef518f6883a8d2f411b3bbffe6ad9', 'e7ff3c68abe10cfa22bc175479e12412d496f113f089c13d4eb6c8c852aed7da', '7d80d9356affbc62f83b60e218d3f277b26c8fbb33049d7e22787be98cad225a']", 'cp_eval:477306932', "path:['58afd4ee5aba3861032b224f8f79fba0112d09945f1b3062579b978475816eab', '442da2ed967c6a85d49a606c3e8910012f1636d597647ebb76aea157c2a155b0', 'ef5cfb6e7756e25f314c0aee2f9abdb0731434a74c9299bbfdd4ad382a5f6306', '07ad3a6152608eb61de38bf641f756331f7e0b39db125957eb869dd3ae101108', 'eb520930528bf0e3defeedcfee1943f70305bffb0f229057a203f6213de88157', '324e2e9041bd96dab1eb01f89cdac3d991afcac681a0f730fe2df26e7e521c61', 'fb9425d16e9685aaa0d8d921ded5ba4c67c72532ea184de6e21009e90c7a0ee8', 'fa5c4c45e6dfdac0c90bb58607206376bde30bacbaf0d61aa402df4e4cb33887', '6d60d16303acb0b6faa80c2a24839a610a8c55e8f4cda4c0afef375879be8d79', '930fc368d72c8e85033e2484cf2dc024b99139ec78544154f836f2f8624754ac', '38ce8aab57b00af6b8f144c50d3738700e2a52e20fa7ff713a65acbea578e95a', 'c856abae3eb16f51a832bf9dd0e10c78aebddae32b93c4015e89fc62951bd224', 'c90db6835ad27b7b088b9de56aad2445378e6354952c43f92b46d30332c2ad4f']", 'layer_first_eval:2669782373', "path:['51b51e3a62d4acf27024383b57f3c6b2bd836781ba4b33959a6ab07e7c80e3eb', 'ad885df843b0e3090bcb0e155c9bacfc38186837304a7a970473684f5af3c1fb', '1de1c798405b3a353cd2be8efc27c1fcc1c0eedabee769ae510c2fdd69899112', 'bf620af97710c6662c2cee19d0b63123b2451e1369f20ccd35885e0b660aa14d', '2725f83a0efe1d16962ddec9aef21a9b0f73ec6b57640e7a14e5733d88f0192c', '7419e68b200294e768f772224603d322efe76b3a5f7ff493515463ca091717a8', '802bd2c84e15c70925ba47b516dd63b8d5e96f759da0fe0eb875d6a76bb809f3', '7ba46110076c41be669c2a40b4441190e2d84c8b3ee42d97567988ec0267a36c', '3c3c2dc3e9b9ab02e02fb3c90097f190e3411f35168b2c119f8334a95c5c2e59', '2ebfd34f62e4d202e8e8eea78d0569c64be2fa15cbda6505d0bc3b158256653e', 'daa1c427339d45b699c11058ae4b3b57c56b9cbda03d6097350cd51acda7b032', '222e84238bc380beae05505dd22b3e36e23b4d0480d0ce3c44c36645fee45da0', '66b44b71b2457bb09615591227be139cbf42e02a18782e4de7931680f16b1703']", 'layer_second_eval:1542099435', "path:['1fa7cc4ec1505e0f7259c0096d0667b01f0021d51bb44596ccbea2abc79dc7ba', 'b2c46405256b3963ed0179d317da94c472b2a0343a6278b9b9a08104bb844224', '478fa556a74eb211273b8dac039493d1bc475f724c9c273b42c05f6ca5e379b2', '18bb4418223737a063b09a2fe23823aa2fd554d4fd284535c0ba5115cf2a0373', '5b931d42bae7cc6eaf7fa681d24f111c3ea436cd93759ac400c71d22cd248ca3', '869002de8818fa80a8bdd10e2dd2dc48df8d65b6b81f233b8b474dadfc8d7b0e', 'fbe27a053022cebe4cbb8194bc2af2027d67dfe03ce6b75350af58191cbeb894', '046d837b1ee9332100d828c054a9a37ef756b25b71d678029b5b5409b1d25fe1', '5c87b8ce0acb3526ebb4075bbcd2f93990d3bd1764745f1a46e394274d71ca7b', '2b76bf599deae8e285e5f7763069b998a9361e32f737d06d95cf0fa28717d152', '88cd3213e03e1e8edc6d8470007497cfd4e9d93566241a3b1854079880f84d0c', '808935676d10a536e39a28f6c00b30610c2f8e1687bbc3b125766b6b6e252e4a', 'a4a4f05b38507befa036d8096351f9046896483b670cb4009777bb57398ddd00']", 'layer_first_eval:254923744', "path:['9ad60ba147ee8ef27f4cf753caa694d5e39384e339c7136e609e6703c7139921', '4bc665882411a0d52b259aea0dd5177acb282cae03f41535c413ab3de73f1d6e', '6c3ec7ae7cdbe36fe43f0d4c09f71eff4eb937cee576d99672fb2ab6093ec672', 'ddbbbb0b3a440e02f7d6600844cfa4b6d25e90cbb71b406f5c2f13a89e6a42ca', '2caa52dfe96e932bbc0c511c0019a6590d4370e05a2fefdbc0375785a1be1a5e', '0c319a177e7c67341dd3b1f362145f49de00db346247ae6c71063157f6d26f6e', '15adbce5704eaf24e6c8f03e025b5f86bc23ffccfe0e1e5dd82f28fe06c2e218', '4df9b0ce3f43f95c33ed2bccc70454cc2ba1c6e0271a145bb3aa6c6aaf67283e', '453e6680bd2ba784540eb7c7a7a296dc9d71f6110eb00c007e311cd1f0a51dee', '5d6b2d4e089c93b652d7d0fe2bfee0d7e7092b3a8843ab6a3b32cf6c5e58e784', 'a156fbd5bd82daa48ccdc36071b7625344c3545bda2add0357cf0e3693319426', '75b0701167193125c48ceceae35f4c411c9aa5ad5fd345d6cf1c246461c6319c']", 'layer_second_eval:3208620180', "path:['6d8c3a2b157bb88004e5eb446052e3bb37ef893edd7c501eb59b141b1c059e2d', '1b36862019331ec2cec35582f6f0f8c33ec92d634e54baafde7b0c4718a8c342', '1a6f0507f33e6580a0722bf3ae81c98a6cadca9fd9d7108459f4835287de8aa9', '5b8cff9f89fc555f9226d3428e432b4b4c8446011626ab5fb5125f2a6ddb9141', 'a6c27a0bb03d8f6c63e9eeb76d3c1bb8bb81d63d68cf3ed61227994bc9ebd457', '7efb66cf5b78d421300159faa5ccfa3044b964336a11566204dfd81dd01cd70e', 'd4a4dc0bfbc74df1b36d9f0f62dc034ac6ab86bedb8aea315db4889a73c9991e', '919394520f1ae4e6bffd2835f214f77c900fa3ede942ddfd9b43200f62e1ef32', 'b8a1c85dd8f020dc94efd5fa23f6d0dcf5b67cbc1b5535c1b7cca524f9336a82', '1dd47bb0ac284966a8c6fedef8382fbc5da88221a8535546d8bc24cfe319f4af', '38e2da1723b9731dc6272441d58f7d42f0d71b16dab43fa7e18defb000b8e39e', '0f310d66046d7c118cb0df430c6bf21f3641962b7501e5e5645d58d9bd201e3d']", 'layer_first_eval:3120628826', "path:['051f0759024eff4083d90b682a13a3b18acb5b9b2d41965b43997a27fc97ceaf', 'fa237eb08bd4b2833045adb3e86a31d8465836dc6a83c225a75e69924dcd35dc', '5eaa7e7e26f237b044dc29b6a4e9a4c240505230df6532439a742c3efcdda7e8', '719fe7837bf854029a633ca38b4c7debf02c34a7cfaef3f78ccf9bd27df29fd4', 'c7712ce424223a319d4e2c6f09b9b814b68b3a682afe2838253d76fb2ae27f78', 'd7462a965e7a0cff448eac4b626692552108d012601def0ce783b7f8132488ad', '23f11e010a445b678bf82155b3ef3d6468faf807573602f5f6c831285971c2cb', 'c787387c0d9cd93a648e2af870d8615ba95bcef33aace94270ec9d39ae44de78', '4845872735bd290cbb9378a8f21191c18fb6d5f3d4477d1c7a8067ab616a3c70', '172d30e9daa7ec125c500808a4bda25aecfe2bab34252fa552ec3cc12ce90f0c', '12a5a4aea15883e396cb57e3fbe55fb1c3963758c3f05dfe5b4bc6a3e5ab13ee']", 'layer_second_eval:2024346897', "path:['586274a9272a82db4a1d1064b966fce26f519b8db34eb5a0cafbca7c0a4e2f12', 'fedeed4bd4038b4aba30748c2647f6f345622a172e8b842be619ddfa7e017da4', '68bc3240d9030c2a327f108c16156d5bbf4c3e9aff241ea573558e8d2098e2f3', 'b1b7110437f586bf484dcc57a144a263013a96fe11c3cfc24ab4ae003b188330', 'b0e67ff2ab415a1ce7a2fb4324d58fd95b1c0d2ef63e047228b10bda8b0f16fd', '7b430a77fc9af8038137f8f1d016311ace1566411e4d93eeec5d485ea27bdcc5', 'f61ccac42e34ee3664bc16b6d825f3606acb6066573314c15f5cf7ad59c382f7', '1fd8f5be35468dc99824e5bfde40c22ec34ef731b177ec1afb4003129d4b30da', '9f923198ff209fa54447b289f74f464338b1fa5c5cf94240e5e6e56e9c902286', 'd9efa84812dc8b2debbaac8a347739e1165ce25d85d40600e546a13fa44f33ac', '53fb63219c80b5dbb697c4589028f1718f59c5408e2c940e9071438f46552412']", 'layer_first_eval:1796730472', "path:['c149d1a83b0fa22e6a304e028e59ab8f6b9709dad968df03b0d96218a81f3e95', '996d8c432e4dff57cf45aebc48daec4c690707f0454c00d4908e44682a992d59', '6056186c4f874352c5ec49d9188293e9647fb43d0d86fce93b938b72223dcedb', '6a9ad2075e9b8c95454030083a220acef47aec5991e7e6bad60991c160fae0a2', '0d365df84280057af5daafdab386f29e33de02df7e8e26546eab806ad1828015', '946a3e43bcfa966e4cecccc5815695dc442f898c06948d77d27e5ee261c3a987', 'c0da82121033ab7bd4dee979b04259e1f04efe954518f11f16576e03cb811337', '3e0ae450568121e47ac9d4ccfb2a2be75656d737fccb395d0202b04760e08a76', '6ff7658cd9ac063cf087ecab8ae531239bfad10f78bbce80d33ae599d4d9436b', '7dc45620dac8c491442099c6e4d812ee363fc0e90bbd83a12d4945b202f432ea']", 'layer_second_eval:1566218616', "path:['92b738049c0778adbe3719b1ca58b17dc93e0e112b30e7bd37ba5e8ccb83d6d7', '58a1d9c6c696a18dab7b185c54037e157c21190fd9521d159cbbe2bcc0838d5f', 'a8c1550f9c38cd0f01ca157d32df0278e16f3e97a640245ffa1a40193de49db0', '3f07b51c095e8625087c0e576a071dd7111003dab0ab2c03418952c8fb45aebf', 'eccf8d48e407b6e83188b03182b5e6804fecfa5dab5ee17cf4cb2cd35a36bfd6', '0051f70f1ccce45a749a9779e1af6c9b565636f45e1fa3173b17474adb41730b', '300454e9c006000ada22bbd043d423875fead8e9cf549005f882db22a72c7bd0', '02cf3d9e99a7409cb96c731a9d49a047cfdf84fbe4adca4622351b8ce50f7dc7', '3c3900c15127f58ddb416cf7f90681e1568494716df11951e2752803317ae0fd', 'b8ca5c278a07b3640e3730d0403cc58301eb50c7b8cc6392a4117f3b0e25b23e']", 'layer_first_eval:1090371103', "path:['6e419261b3777ae96bc292861ec006d88757feed6c8e90f4e11689821f0ae2d4', '81f3f138dd449a8d9acc052a786bfd5d8df676cd5f9f4cc92e36ac2f303f2277', '81b54cd20478834e2a60283acc16b51180292310171520c0af2bf5fd02961afa', '5e4fb3a136889f8a6fb42768f6f74e7230262282ebb32e79306ea5d00736b1a8', '4ed4e46a4a0b971159eaed0de0748743ea0bf2fedbb65ca52612b59e07a094b1', 'c9f67e7c5f6bf1a3c7ff2936aaa4d571918c1176c82d4d7af1866d42cfe31824', '92edbd1a45b37777c20e176cba02cffa5375d1b3ee85c8ea262077338777d74d', 'e659e1f4754bebadeef8427da781115762ea0a4f0ae74526c03a48bd3886ca28', '0910186530b067582f78578111db616664d7514d4fe86110c7ac9f60d58b5b90']", 'layer_second_eval:698494644', "path:['73c8a277b91be7ad4ab321f5b6d00f9f2421f1b4d5986a52f02901169c45ea2c', '8eec09604717ed891fb8cc7b59e6e2b4886ceacd73d16f50d51b8f2023fc6cb3', '81ea31ec79c1b780ec79444d4df41ddf8907370f0febd016296157ce86ed3b3f', 'f50853e73a606d0832ef28b9735efd7fe2ffd41130312d01a8c5a29bb523f460', '7e1f240609e082f5653da9cfb38c28ccd383aa80248b2cd6a8262e2b1928aabb', '9f9292ccfe8c80c8ce403ab50c0fdd8e83f8459a8b356165eb0733566f04f676', '34f6cdd303b316f16bf3a56d608877738b9a8078b222ca49a981759b2b75c421', 'dbc5231870b48a5085be42206fa309b03a118dc359170262eba669903a3cb0a3', 'cfc1202c6c87b02348ca6f07628d0aea012cda848301323904821745389cf6cd']", 'layer_first_eval:2807155706', "path:['86790e286ab490dfca86bba98f50fbab2e698096391d38e0e00e64f9d357eddc', 'a5f5fd78f078a6a4a00cf7ce30bc1bf0349c3c4637e41b98f02ee71fc4493711', '3493923c553948d213a4dc42bad6e85fa7e9a1b1982114fddce7a82cc7869927', 'c109266512dfb1b9cabc2dacdc8088e007dee5e5c27a6486874e3741a4b86b35', '61f1de18a01559a27130784ad860814b915246057d520de834363960ca8afeaa', '775690609a5828c51dfbee7671d865a9043d8fa80d834d4355d59f28d762248b', '166db15aeafa1f3900a30b6d32327f469fe4d93c92cc06213074914df46d2bdf', '8ae42a962801794166bfedd43f51240447348db7b27ea30872dbb3967372f4ca']", 'layer_second_eval:1197830162', "path:['c70611b29a0d624a41a54aa806079badcb54a6e70ce7ff94277ee802cab7db25', '5c842ca3b2736749c5f41d9c9a0358d5ee581f9a5c546664c13df51881895c0c', 'b6591f8fbc87a1632a55b962157628912628d0fa845eeb271c5e639d216f275a', '1d40981c444a26c21a464acbf4c2881944fde83a8f92d9fb16dac888eb940bdf', '245a6428099092d86e4e24963f809a7fe3a39da1532a5561b4f48baf3b05b995', '6578b29c242bf875ed7686532b1a860014ba3522ee3d5abbcec8d83990ca6005', 'ae084be3e8efe4e40f0234819851e917fdc339f7c50f9231dab88e68613e042f', '02e070169662eeef4764985ff525d891779958948474d26098f178a68ec32ae8']", 'layer_first_eval:1504506154', "path:['3fe19c356d49bf6ecbb6ebecd0856b5869d8fddb0cfc4daa97aab3e41b17e80e', '921b2af1f6b3a7bdb73dc1c008e9f946008f8a9883a496ede8df21b9f14c4394', 'fe03c6e1d9f865fe3f93591c889c84275b3f4aeef1bab2af647835e09543d662', 'ce2939b339ce4471e9deb3403022126644bb4dad40feb06363c44ef79c9c23ee', '9971c1d1e2d3276665d2f105fe590aef2aab92e79f8b79fcf5835eaff8007f44', '479492672ed296c5f0ddced0a4fa2c26264772772d1c7c1bc55052e0c955c4de', 'cb7fa843377ae7575a2943eacd482c81ec5356a323f090b5b7c2d0789d77ba12']", 'layer_second_eval:334168867', "path:['135002e85a5f4a5c6e430251903f82b5b0db0545dfa805cbf16a7d80bf0d68a8', '4ca95d2cdb101d1a80ff4dda280bf546c4f05030e96b1b1466a734b05f595c0d', '30094468ce74a362803054e1b942db387acb16006e01dadff86c55db09155aa2', '1c2e38c98f2db89174848aaa14171497facae8233a4b46b9dacd72fa808b6a15', 'ff7e6ef7d14eeede3d95027b3aeaff1328edca32365d5a602cef07ccf2076d13', '6795e0ef4336aa07d99dfdff3591ee2839dfc8349bfaca193b209cc47f66f1ae', '69126f97e4d83d47fd4ec94fad9de2ef70edff0745d10fd22399937ef89e58db']", 'layer_first_eval:205671584', "path:['4dd03982880e7dbe7abf43984fb7910a52164b87935de65c3a21ca38cd45bf86', 'c7b731864c3fdef669aeb921c6f3970eb1b7d6db382b4af2050a1c52ca1ec8b1', '277db5f7db92c6a004c5371f95a9e026662912dd229b5ee4d36804e4ebf0ae10', 'c8ffff94eee04bd5a0b45200bd7b999c6f5e001852f1a451ed9109518c224553', '9ce5714b43360c3176e3704ba48488bafccc44db68451c080df73ef3bb76b503', 'b68d524eaf64bb35f5a56c6c68c357ef6204afe8856ea9821accbc158385b8ad']", 'layer_second_eval:796933781', "path:['fe83a71a16c6e25d6b4e429590a33c18d293de7e05d7ee7db20bdfafea48152c', '3a4aeb4c54de6b7b08e5104c5d3e4aa50e162b8c42920cd8bc2e2fd8db0c3a32', 'a224e77b9daa4042f6e409e1c84112000804c239b81d67e978e814287df1cfce', 'ebf885588daaebceb511a92a66de6b4f296fcd95984f2e4e6549c0f89224b683', '0c92cc27ee8200d2451314ffe2801f98a5fff713e3067458fc4d9134de4ef47e', '6bd2c8259f7fd8a5586580d355001c869ccf1f42a2d835b100bc0e1bb8aadfaa']", 'layer_first_eval:1172723434', "path:['95aedc72d410c11f8ee1d4b1d9d953e29c6d91a6052fa3dd94cf9cf844c295a4', 'c1c514f592cbdd4ceebc615c5d408630428f78904f5574cdab7cbea356baf621', '8fbee6f3276977dc678ee1cff10b077a3090c1804a41677a5b7783122bac7d82', '6f2be99f8245e4609a4cb04386c9f5b7c5043437da931521eea8b71e7ea6622e', 'e608b7ba7be420932014fba06c1d7184a336c76248fbc7192c52581bcf423c93']", 'layer_second_eval:757797237', "path:['f0c3739db1b03bfbe6a5484fc50693e874a9b7eb33cddd8fa2bbe68d8d3e06c4', '16a99ddd4161d2e7409969400b69e104b97d840cb56ef9887c2156c1ecc892bb', '0b54f6a899492456c33dfd3f8c50d0e1f2aeba9afebe76c9bb7269bf85ed6f22', '871178c50ee7a3080261459595187dc7bc26293f70faa1d18e4eea38628e4025', '140db42fdf3e2ec6c8c1e6b085330a7d54d8e0ed55f6bb35513fb9d5388a5f24']", 'layer_first_eval:2917702132', "path:['171e932578639b4a6c4a4161433c04e007096d33dd54129a36468fb02d684f3e', '1c361b58b1693492241e7895efe5c410475eb9f65fafae0160c65e16f63e51fb', '27c6c37aa47a569dbb4804e91fc00e0d07d69adffd5ad30fab524d5628f6e23b', 'd5386d3561e46aa598c7d376322fd26eb260a1b6ddb75f4b2bdee8cb5ad3e5cc']", 'layer_second_eval:2509621709', "path:['316db770044d27dbf2e7af7e31e886274fdbd3eb591d954cedebb00410e771e1', '83f551aae85628c86df0b4da79f6ecd7701747498c4f9bdc65054c264ed0f235', '43459233e43ca5625bc6a6580677cb8ff2e746f3edde656d59f9f39e2115c2a4', 'bcc6e5e80a7789698f7f3c1877f045bb2cc7aa652296a45419ce5bcdad3de2ba']"]